In [1]:
#%load_ext nb_black

In [2]:
import sys

sys.path.append("../")
import yaml
import logging
import requests
from pprint import pprint
import json
import time
import os
from pymongo import MongoClient
import dateparser
import psutil
from nlp_utils import NLPUtils
from lda_module import LdaModule

In [3]:
mongourl = "mongodb://localhost:27017/"
MONGO_CLIENT = MongoClient(mongourl)
LANG_CODE = 'nl'

In [4]:
collection = MONGO_CLIENT["news"]["article" + "_" + LANG_CODE]
not_processed_docs = collection.find()
# for doc in not_processed_docs:
#    print(doc)
#    time.sleep(10)

In [5]:
training_set = []
i = 0
for document in not_processed_docs:
    if i % 10000 == 0:
        print(i)
        print(psutil.virtual_memory())
    training_set.append(document['text'])
    i += 1

0
svmem(total=8589934592, available=3141836800, percent=63.4, used=4097929216, free=545914880, active=1749180416, inactive=2241425408, wired=2348748800)
10000
svmem(total=8589934592, available=3031863296, percent=64.7, used=4207144960, free=431042560, active=1841582080, inactive=2244763648, wired=2365562880)
20000
svmem(total=8589934592, available=2797834240, percent=67.4, used=4341575680, free=187084800, active=1975648256, inactive=2250518528, wired=2365927424)
30000
svmem(total=8589934592, available=2760675328, percent=67.9, used=4377681920, free=147648512, active=2011512832, inactive=2251399168, wired=2366169088)
40000
svmem(total=8589934592, available=2591199232, percent=69.8, used=4545417216, free=15335424, active=2194051072, inactive=2254667776, wired=2351366144)
50000
svmem(total=8589934592, available=2604851200, percent=69.7, used=4548964352, free=68800512, active=2197225472, inactive=2255437824, wired=2351738880)
60000
svmem(total=8589934592, available=2461089792, percent=71.3

In [6]:
# Load custom stop words from file
with open('custom_stop_words.json', 'r') as sw:
    custom_stop_words = json.load(sw)
custom_stop_words = custom_stop_words['s_words']

In [7]:
nlp_utils = NLPUtils(lang=LANG_CODE)

In [8]:
# Some preparation before running LDA
doc_collection = []
print("Parsing articles...")
i = 0
for doc in training_set[:1000]:
    if i % 100 == 0: print(i)
    tokens = nlp_utils.parse_text(doc)
    doc_collection.append(tokens)
    i += 1

Parsing articles...
0
100
200
300
400
500
600
700
800
900


In [9]:
print("Completed parsing {} articles".format(len(doc_collection)))

num_docs = len(doc_collection)
num_topics = 20
lda = LdaModule(lang = LANG_CODE, num_docs = num_docs, doc_collection = doc_collection, num_topics = num_topics, trained = False)
lda.runLDA()

docs_topics_dict = lda.get_docs_topics_dict()

Completed parsing 1000 articles
... Building dictionary ...
... Finding collocations ...
... Building corpus ...
... Building LDA model ...
... Retrieving topics ...


In [10]:
# == Saving model checkpoint ==
lda.save_LDA_model()